In [1]:
from __future__ import division
import argparse

from PIL import Image
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt

import sys

from gym_unity.envs import UnityEnv

%matplotlib inline

print("Python version:")
print(sys.version)

# check Python version
if (sys.version_info[0] < 3):
    raise Exception("ERROR: ML-Agents Toolkit (v0.3 onwards) requires Python 3")


Python version:
3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [3]:
class BallVecProcessor(Processor):
    def process_action(self, action):
        #print(action)
        action = [np.floor(action/21)*((action-20)/10-1.1), (1-np.floor(action/21))*((action-20)/10+1)]
        #print(action)
        return action
    def process_info(self, info):
        key, value = info.items()
        #print(key)
        #print(value)
        #print(value[1].rewards)
        key = 1
        value = value[1].rewards
        info = {key: value}
        
        """Processes the info as obtained from the environment for use in an agent and
        returns it.

        # Arguments
            info (dict): An info as obtained by the environment

        # Returns
            Info obtained by the environment processed
        """
        return info

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)
    

In [4]:
env_name = "mlagents/envs/3DBall_128"  # Name of the Unity environment binary to launch
env = UnityEnv(env_name, worker_id=0, use_visual=False)

nb_actions = 42
print(str(env))

INFO:mlagents.envs:
'Ball3DAcademy' started successfully!
Unity Academy name: Ball3DAcademy
        Number of Brains: 1
        Number of External Brains : 1
        Reset Parameters :
		
Unity brain name: Ball3DBrain
        Number of Visual Observations (per agent): 1
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): [2]
        Vector Action descriptions: , 
INFO:gym_unity:1 agents within environment.


<UnityEnv instance>


In [5]:
# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 42)                714       
_________________________________________________________________
activation_3 (Activation)    (None, 42)                0         
Total para

In [6]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
processor = BallVecProcessor()
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=42, memory=memory, nb_steps_warmup=1000,
               target_model_update=1e-2, policy=policy, processor = processor)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [7]:
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=100000, visualize=False, verbose=2)

Training for 100000 steps ...
    20/100000: episode: 1, duration: 0.545s, episode steps: 20, steps per second: 37, episode reward: 0.900, mean reward: 0.045 [-1.000, 0.100], mean action: -0.055 [-1.000, 1.000], mean observation: -0.313 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
    43/100000: episode: 2, duration: 0.236s, episode steps: 23, steps per second: 97, episode reward: 1.200, mean reward: 0.052 [-1.000, 0.100], mean action: 0.048 [-1.000, 0.700], mean observation: -0.256 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
    60/100000: episode: 3, duration: 0.200s, episode steps: 17, steps per second: 85, episode reward: 0.600, mean reward: 0.035 [-1.000, 0.100], mean action: -0.071 [-0.900, 0.800], mean observation: -0.655 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
    83/100000: episode: 4, duration: 0.261s, episode steps: 23, steps per second: 88, episode reward: 1.200, mean reward: 0.052 [-1.000, 0.100], mean action: -0.

   710/100000: episode: 32, duration: 0.213s, episode steps: 21, steps per second: 99, episode reward: 1.000, mean reward: 0.048 [-1.000, 0.100], mean action: 0.086 [-0.900, 1.000], mean observation: -0.288 [-6.867, 4.000], loss: --, mean_absolute_error: --, mean_q: --
   737/100000: episode: 33, duration: 0.295s, episode steps: 27, steps per second: 91, episode reward: 1.600, mean reward: 0.059 [-1.000, 0.100], mean action: 0.026 [-1.000, 0.900], mean observation: -0.428 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
   755/100000: episode: 34, duration: 0.172s, episode steps: 18, steps per second: 104, episode reward: 0.700, mean reward: 0.039 [-1.000, 0.100], mean action: 0.017 [-0.900, 1.000], mean observation: -0.411 [-7.848, 4.000], loss: --, mean_absolute_error: --, mean_q: --
   777/100000: episode: 35, duration: 0.225s, episode steps: 22, steps per second: 98, episode reward: 1.100, mean reward: 0.050 [-1.000, 0.100], mean action: 0.014 [-1.000, 1.000], mean ob

  1455/100000: episode: 63, duration: 0.274s, episode steps: 21, steps per second: 77, episode reward: 1.000, mean reward: 0.048 [-1.000, 0.100], mean action: 0.074 [-0.900, 1.000], mean observation: -0.245 [-6.867, 4.000], loss: 0.128503, mean_absolute_error: 1.179777, mean_q: 1.618593
  1477/100000: episode: 64, duration: 0.239s, episode steps: 22, steps per second: 92, episode reward: 1.100, mean reward: 0.050 [-1.000, 0.100], mean action: 0.032 [-1.000, 1.000], mean observation: -0.274 [-7.848, 4.000], loss: 0.136981, mean_absolute_error: 1.262225, mean_q: 1.719616
  1496/100000: episode: 65, duration: 0.210s, episode steps: 19, steps per second: 90, episode reward: 0.800, mean reward: 0.042 [-1.000, 0.100], mean action: 0.037 [-0.900, 1.000], mean observation: 0.344 [-7.848, 4.000], loss: 0.123840, mean_absolute_error: 1.310185, mean_q: 1.775242
  1516/100000: episode: 66, duration: 0.254s, episode steps: 20, steps per second: 79, episode reward: 0.900, mean reward: 0.045 [-1.000,

  2089/100000: episode: 92, duration: 0.244s, episode steps: 19, steps per second: 78, episode reward: 0.800, mean reward: 0.042 [-1.000, 0.100], mean action: 0.071 [-1.000, 1.000], mean observation: -0.282 [-7.848, 4.000], loss: 0.245230, mean_absolute_error: 2.774495, mean_q: 3.376879
  2107/100000: episode: 93, duration: 0.197s, episode steps: 18, steps per second: 91, episode reward: 0.700, mean reward: 0.039 [-1.000, 0.100], mean action: -0.011 [-0.900, 0.800], mean observation: 0.061 [-7.848, 4.000], loss: 0.250006, mean_absolute_error: 2.801099, mean_q: 3.406243
  2125/100000: episode: 94, duration: 0.213s, episode steps: 18, steps per second: 85, episode reward: 0.700, mean reward: 0.039 [-1.000, 0.100], mean action: 0.089 [-1.000, 0.900], mean observation: 0.134 [-7.848, 4.000], loss: 0.301995, mean_absolute_error: 2.804128, mean_q: 3.436416
  2144/100000: episode: 95, duration: 0.237s, episode steps: 19, steps per second: 80, episode reward: 0.800, mean reward: 0.042 [-1.000,

  2731/100000: episode: 121, duration: 0.459s, episode steps: 27, steps per second: 59, episode reward: 1.600, mean reward: 0.059 [-1.000, 0.100], mean action: -0.078 [-1.000, 0.900], mean observation: -0.290 [-7.848, 4.000], loss: 0.372216, mean_absolute_error: 4.362435, mean_q: 5.118945
  2751/100000: episode: 122, duration: 0.306s, episode steps: 20, steps per second: 65, episode reward: 0.900, mean reward: 0.045 [-1.000, 0.100], mean action: -0.108 [-1.000, 0.800], mean observation: -0.374 [-7.848, 4.000], loss: 0.371143, mean_absolute_error: 4.453413, mean_q: 5.227214
  2775/100000: episode: 123, duration: 0.447s, episode steps: 24, steps per second: 54, episode reward: 1.300, mean reward: 0.054 [-1.000, 0.100], mean action: 0.006 [-1.000, 0.900], mean observation: 0.173 [-7.848, 4.000], loss: 0.319024, mean_absolute_error: 4.324211, mean_q: 5.050910
  2793/100000: episode: 124, duration: 0.388s, episode steps: 18, steps per second: 46, episode reward: 0.700, mean reward: 0.039 [-

...

 76703/100000: episode: 785, duration: 0.747s, episode steps: 65, steps per second: 87, episode reward: 5.400, mean reward: 0.083 [-1.000, 0.100], mean action: 0.093 [-1.000, 1.000], mean observation: 0.077 [-6.946, 4.000], loss: 5.493799, mean_absolute_error: 90.108566, mean_q: 94.534683
 76925/100000: episode: 786, duration: 2.546s, episode steps: 222, steps per second: 87, episode reward: 21.100, mean reward: 0.095 [-1.000, 0.100], mean action: 0.006 [-1.000, 1.000], mean observation: 0.203 [-6.867, 4.000], loss: 5.928614, mean_absolute_error: 89.686546, mean_q: 94.039124
 77505/100000: episode: 787, duration: 6.762s, episode steps: 580, steps per second: 86, episode reward: 56.900, mean reward: 0.098 [-1.000, 0.100], mean action: 0.000 [-1.000, 1.000], mean observation: 0.145 [-6.867, 4.000], loss: 8.002007, mean_absolute_error: 88.640610, mean_q: 93.060844
 77535/100000: episode: 788, duration: 0.381s, episode steps: 30, steps per second: 79, episode reward: 1.900, mean reward: 0.

 80881/100000: episode: 814, duration: 0.493s, episode steps: 24, steps per second: 49, episode reward: 1.300, mean reward: 0.054 [-1.000, 0.100], mean action: 0.060 [-1.000, 1.000], mean observation: -0.417 [-6.867, 4.000], loss: 7.507509, mean_absolute_error: 81.198334, mean_q: 85.241707
 80908/100000: episode: 815, duration: 0.387s, episode steps: 27, steps per second: 70, episode reward: 1.600, mean reward: 0.059 [-1.000, 0.100], mean action: -0.063 [-1.000, 1.000], mean observation: -0.198 [-6.867, 4.000], loss: 7.777591, mean_absolute_error: 80.926270, mean_q: 84.894707
 81165/100000: episode: 816, duration: 3.184s, episode steps: 257, steps per second: 81, episode reward: 24.600, mean reward: 0.096 [-1.000, 0.100], mean action: -0.009 [-1.000, 1.000], mean observation: 0.114 [-6.867, 4.000], loss: 7.158630, mean_absolute_error: 80.639664, mean_q: 84.769371
 81232/100000: episode: 817, duration: 0.893s, episode steps: 67, steps per second: 75, episode reward: 5.600, mean reward: 

 82994/100000: episode: 843, duration: 0.333s, episode steps: 32, steps per second: 96, episode reward: 2.100, mean reward: 0.066 [-1.000, 0.100], mean action: -0.183 [-1.000, 0.900], mean observation: 0.275 [-6.867, 4.000], loss: 4.316245, mean_absolute_error: 76.868958, mean_q: 81.026703
 83115/100000: episode: 844, duration: 1.370s, episode steps: 121, steps per second: 88, episode reward: 11.000, mean reward: 0.091 [-1.000, 0.100], mean action: 0.030 [-1.000, 1.000], mean observation: 0.056 [-7.848, 4.000], loss: 7.702164, mean_absolute_error: 76.815720, mean_q: 80.844482
 83141/100000: episode: 845, duration: 0.304s, episode steps: 26, steps per second: 86, episode reward: 1.500, mean reward: 0.058 [-1.000, 0.100], mean action: -0.065 [-1.000, 1.000], mean observation: -0.550 [-6.867, 4.000], loss: 8.619825, mean_absolute_error: 76.150185, mean_q: 80.220253
 83309/100000: episode: 846, duration: 1.935s, episode steps: 168, steps per second: 87, episode reward: 15.700, mean reward:

 88065/100000: episode: 872, duration: 5.290s, episode steps: 464, steps per second: 88, episode reward: 45.300, mean reward: 0.098 [-1.000, 0.100], mean action: -0.008 [-1.000, 1.000], mean observation: 0.188 [-6.867, 4.000], loss: 6.892467, mean_absolute_error: 72.531960, mean_q: 76.825165
 88135/100000: episode: 873, duration: 0.781s, episode steps: 70, steps per second: 90, episode reward: 5.900, mean reward: 0.084 [-1.000, 0.100], mean action: 0.044 [-1.000, 1.000], mean observation: 0.203 [-6.867, 4.000], loss: 9.304734, mean_absolute_error: 72.021996, mean_q: 76.263512
 88621/100000: episode: 874, duration: 5.299s, episode steps: 486, steps per second: 92, episode reward: 47.500, mean reward: 0.098 [-1.000, 0.100], mean action: -0.013 [-1.000, 1.000], mean observation: 0.146 [-7.848, 4.000], loss: 6.594003, mean_absolute_error: 72.318176, mean_q: 76.635223
 89135/100000: episode: 875, duration: 5.654s, episode steps: 514, steps per second: 91, episode reward: 50.300, mean reward

 93862/100000: episode: 901, duration: 0.807s, episode steps: 67, steps per second: 83, episode reward: 5.600, mean reward: 0.084 [-1.000, 0.100], mean action: -0.131 [-1.000, 1.000], mean observation: 0.113 [-6.867, 4.000], loss: 11.123363, mean_absolute_error: 71.133453, mean_q: 75.843323
 94121/100000: episode: 902, duration: 2.993s, episode steps: 259, steps per second: 87, episode reward: 24.800, mean reward: 0.096 [-1.000, 0.100], mean action: -0.018 [-1.000, 1.000], mean observation: 0.240 [-6.867, 4.000], loss: 7.026450, mean_absolute_error: 72.036125, mean_q: 76.875580
 94964/100000: episode: 903, duration: 9.783s, episode steps: 843, steps per second: 86, episode reward: 83.200, mean reward: 0.099 [-1.000, 0.100], mean action: -0.000 [-1.000, 1.000], mean observation: 0.182 [-7.848, 4.000], loss: 7.735035, mean_absolute_error: 72.468460, mean_q: 77.279922
 95263/100000: episode: 904, duration: 3.435s, episode steps: 299, steps per second: 87, episode reward: 28.800, mean rewa

In [8]:
ENV_NAME = '3DBall_128'
# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_vec_weights.h5f'.format(ENV_NAME), overwrite=True)

# Finally, evaluate our algorithm for 5 episodes.
dqn.test(env, nb_episodes=1000, visualize=True)

Testing for 1000 episodes ...
Episode 1: reward: 100.100, steps: 1001
Episode 2: reward: 100.100, steps: 1001
Episode 3: reward: 100.100, steps: 1001
Episode 4: reward: 100.100, steps: 1001
Episode 5: reward: 100.100, steps: 1001
Episode 6: reward: 100.100, steps: 1001
Episode 7: reward: 100.100, steps: 1001
Episode 8: reward: 100.100, steps: 1001
Episode 9: reward: 100.100, steps: 1001
Episode 10: reward: 100.100, steps: 1001
Episode 11: reward: 100.100, steps: 1001
Episode 12: reward: 100.100, steps: 1001
Episode 13: reward: 100.100, steps: 1001
Episode 14: reward: 100.100, steps: 1001
Episode 15: reward: 100.100, steps: 1001
Episode 16: reward: 100.100, steps: 1001
Episode 17: reward: 100.100, steps: 1001
Episode 18: reward: 100.100, steps: 1001
Episode 19: reward: 100.100, steps: 1001
Episode 20: reward: 100.100, steps: 1001
Episode 21: reward: 100.100, steps: 1001
Episode 22: reward: 100.100, steps: 1001
Episode 23: reward: 100.100, steps: 1001
Episode 24: reward: 100.100, steps: 

KeyboardInterrupt: 